In [ ]:
Loading packages

In [2]:
from sec_api import QueryApi, ExtractorApi
import csv
import json
from datetime import datetime
queryApi = QueryApi(api_key="b12062d7e9c95f42d2cee3a526aacc7eb1980f5e8da55254010c1bb88eb6da80")
extractorApi = ExtractorApi("b12062d7e9c95f42d2cee3a526aacc7eb1980f5e8da55254010c1bb88eb6da80")

#Step 1. Using SEC API to get links of 10-K files between 2000 and 2018 

In [ ]:
jsons=[]
for i in range(50):
    #loading 10-K files issued between 2000-01-01 to 2018-12-31
    query = {
        "query": { "query_string": { 
            "query": "filedAt:{2000-01-01 TO 2018-03-31} AND formType:\"10-K\"" 
        } },
        "from": 200*i,
        "size": "100000000000000000000000000000",
        "sort": [{ "filedAt": { "order": "desc" } }]
    }
    filings = queryApi.get_filings(query)
    jsons.append(filings)
    print("Iteration",i,"is finished!")



#Step 2. write links to the "raw_data.csv" file

In [ ]:
f=open('raw_data.csv','a',newline='')
csvwriter=csv.writer(f)
count=0

for filings in jsons:
    filings=filings['filings']
    for filing in filings:
        name=filing['companyName']; ticker=filing['ticker']; cik=filing['cik']; formtype=filing['formType']; date=filing['filedAt']; url=filing['linkToFilingDetails'] 
        json={'Company Name':name, 'Ticker':ticker, 'CIK': cik, 'formType':formtype, 'Filing Date': date, 'link to Filing Details':url}
    
        if count==0:
            header=json.keys()
            csvwriter.writerow(header)
            count+=1
        csvwriter.writerow(json.values())
        count+=1
f.close()
print(count-1)


1000


#Step 3. loading AAER issuing data (AAER no, CIK, Year)

In [ ]:
f=open('AAER_firm_year.csv','r')
csvreader=csv.reader(f)
count=0
cik_years=[]
for row in csvreader:
    #check if the row is the header
    if count==0:
        count+=1
        continue
    cik=row[1]
    year=row[2]
    cik_years.append((cik,year))
print(len(cik_years))

1746


#Step 4. Extracting MD&A section from the 10-K file links of fraud cases

In [ ]:
f=open('raw_data.csv','r')
csvreader=csv.reader(f)
count=0; dicts=[]
for row in csvreader:
    #check if the formtype is 10-K:
    if row[3] !="10-K":
        continue
    
    #Initialize the dict
    name=row[0]; ticker=row[1]; cik=row[2]; formtype=row[3]; year=row[4][0:4]; url=row[5]
    cik_year=(cik,year)

    if cik_year not in cik_years:
        continue
    if cik_year in cik_years:
        isFraud=1
    else:
        isFraud=0
    section_text=extractorApi.get_section(url,"7","text")
    data={'Company Name':name, 'Ticker':ticker, 'CIK': cik, 'formType':formtype, 'Filing Year': year, 'isFraud':isFraud, 'MD&A':section_text}
    dicts.append(data)
    count+=1
f.close()


write data as json to file

In [ ]:
import json
with open('fraud_case.json','w') as fp:
    json.dump(dicts,fp,indent=7)


#Step 5. Extracting MD&A section from the 10-K file links of non-fraud cases

In [ ]:
f=open('raw_data.csv','r')
csvreader=csv.reader(f)
count=0; dicts=[]
for row in csvreader:
    #check if the formtype is 10-K:
    if row[3] !="10-K":
        continue
    
    #Initialize the dict
    name=row[0]; ticker=row[1]; cik=row[2]; formtype=row[3]; year=row[4][0:4]; url=row[5]
    cik_year=(cik,year)

    if cik_year in cik_years:
        continue
    if cik_year in cik_years:
        isFraud=1
    else:
        isFraud=0
    section_text=extractorApi.get_section(url,"7","text")
    data={'Company Name':name, 'Ticker':ticker, 'CIK': cik, 'formType':formtype, 'Filing Year': year, 'isFraud':isFraud, 'MD&A':section_text}
    dicts.append(data)
    count+=1

    if count==350:
        break
f.close()

In [ ]:
import json
with open('non-fraud_case.json','w') as fp:
    json.dump(dicts,fp,indent=7)
